# FUNDAMENTALS_21. Tensorflow

  - 우리가 실습에서 많이 사용한 딥러닝 프레임워크는 Tensorflow였다.
  - 하지만 좋은 딥러닝 프레임워크는 Tensorflow 말고도 pyTorch라는 프레임워크도 있다.
  - Tensorflow와 pyTorch 둘 다 서로 가진 장점들을 흡수하면서 더 좋은 딥러닝 프레임워크로 진화하고 있다.

## Tensorflow2 API 알아보기
  - tensorflow에서 딥러닝 모델을 작성하는 방법은 크게 3가지가 존재한다.
  - Sequential / Functional / Model subclassing 이렇게 3가지가 존재한다.

### Tensorflow2 Sequential Model
  - 앞선 자료들에서 공부했던 모델은 대부분 Sequential 모델이었다.
  - model = keras.Sequential( )로 선언해주고 모델을 활용하면 손쉽게 딥러닝 모델을 쌓아갈 수 있다.
  - 가장 큰 장점은 초보자가 접근하기에 매우 쉽다는 것이다.
  - 하지만 모델의 입력과 출력이 여러개인 경우에는 사용하기 어렵다.
  - 그래서 Sequential 모델은 반드시 입력 1가지, 출력 1가지를 전제로 하고 사용한다.

In [ ]:
# Sequential Model 코드

import tensorflow as tf
from tensorflow import keras

model = keras.Sequential()
model.add(__넣고싶은 레이어__)
model.add(__넣고싶은 레이어__)
model.add(__넣고싶은 레이어__)

model.fit(x, y, epochs=10, batch_size=32)

### Tensorflow2 Functional API
  - Functional API를 활용하면 Sequential Model을 활용하는 것보다 더 자유로운 모델링을 진행할 수 있다.
  - Functional API는 keras.Model을 사용한다.
  - Sequential Model은 keras.Model을 상속받아 확장한 특수 사례인데 Functional API는 더 직관적이라고 생각한다.
  - Functional API는 입력과 출력을 규정함으로써 모델 전체를 규정한다. 그래서 input이 될 수 있는 텐서가 여러개일 수 있고 레이어들을 자유롭게 엮어 출력을 규정할 수 있다.
  - Sequential Model의 제약점이 1개의 입/출력이었다면 Functional API는 다중 입력/출력을 가지는 모델이다.

In [ ]:
# Functional API 코드

import tensorflow as tf
from tensorflow import keras

inputs = keras.Input(shape=(__원하는 입력값 모양__))
x = keras.layers.__넣고싶은 레이어__(관련 파라미터)(input)
x = keras.layers.__넣고싶은 레이어__(관련 파라미터)(x)
outputs = keras.layers.__넣고싶은 레이어__(관련 파라미터)(x)

model = keras.Model(inputs=inputs, outputs=outputs)
model.fit(x,y, epochs=10, batch_size=32)

### Tensorflow2 Subclassing
  - subclassing 역시 keras.Model을 상속받은 모델 클래스를 만드는 것이다.
  - keras.Model은 __init__ 메서드 안에서 레이어 구성을 정의한다.
  - 그리고 call( ) 이라는 메서드 안에서 레이어 간 forward propagation을 구현한다.

In [ ]:
# Subclassing 코드

import tensorflow as tf
from tensorflow import keras

class CustomModel(keras.Model):
    def __init__(self):
        super(CustomModel, self).__init__()
        self.__정의하고자 하는 레이어__()
        self.__정의하고자 하는 레이어__()
        self.__정의하고자 하는 레이어__()
    
    def call(self, x):
        x = self.__정의하고자 하는 레이어__(x)
        x = self.__정의하고자 하는 레이어__(x)
        x = self.__정의하고자 하는 레이어__(x)
        
        return x
    
model = CustomModel()
model.fit(x,y, epochs=10, batch_size=32)

### Sequential을 활용해서 MNIST 문제 해결

In [2]:
# 라이브러리 및 데이터 구성하기

import tensorflow as tf
from tensorflow import keras
import numpy as np

# 데이터 구성부분
mnist = keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train=x_train[...,np.newaxis]
x_test=x_test[...,np.newaxis]

print(len(x_train), len(x_test))

60000 10000


In [5]:
# Sequential Model 구성

model = keras.Sequential([
    keras.layers.Conv2D(32, 3, activation='relu'),
    keras.layers.Conv2D(64, 3, activation='relu'),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

# 모델 학습 설정

model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test, y_test, verbose=2)

Epoch 1/5
1875/1875 [==============================] - 31s 3ms/step - loss: 0.1050 - accuracy: 0.9673
Epoch 2/5
1875/1875 [==============================] - 7s 3ms/step - loss: 0.0330 - accuracy: 0.9899
Epoch 3/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0192 - accuracy: 0.9938
Epoch 4/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0121 - accuracy: 0.9959
Epoch 5/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0092 - accuracy: 0.9970
313/313 - 1s - loss: 0.0494 - accuracy: 0.9865


[0.049376267939805984, 0.9865000247955322]

### Functional API 활용
  - Sequential Model과 큰 차이는 없지만 이번에는 직접 keras.Model을 활용해야 하므로 input 및 output 레이어 구성을 통해 model을 구현해야 한다.

In [6]:
# 라이브러리 및 데이터 구성하기

import tensorflow as tf
from tensorflow import keras
import numpy as np

# 데이터 구성부분
mnist = keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train=x_train[...,np.newaxis]
x_test=x_test[...,np.newaxis]

print(len(x_train), len(x_test))

60000 10000


In [15]:
# Functional API 구성

inputs = keras.Input(shape=(28, 28, 1))

x = keras.layers.Conv2D(32, 3, activation='relu')(inputs)
x = keras.layers.Conv2D(64, 3, activation='relu')(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(128, activation='relu')(x)
predictions = keras.layers.Dense(10, activation='softmax')(x)

model = keras.Model(inputs=inputs, outputs=predictions)

In [16]:
# 모델 학습 설정

model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test, y_test, verbose=2)

Epoch 1/5
1875/1875 [==============================] - 7s 3ms/step - loss: 0.1047 - accuracy: 0.9684
Epoch 2/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0355 - accuracy: 0.9886
Epoch 3/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0178 - accuracy: 0.9942
Epoch 4/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0135 - accuracy: 0.9953
Epoch 5/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0088 - accuracy: 0.9973
313/313 - 1s - loss: 0.0513 - accuracy: 0.9872


[0.051265742629766464, 0.9872000217437744]

### Subclassing 활용
  - subclassing 방법은 keras.Model을 상속받은 클래스를 만들어야 한다.
  - __init__ 메서드 안에서 레이어를 선언하고 call 메서드 안에서 forward propagation을 구현하는 방식이다.
  - Functional 방식과 비교하면 call의 입력이 input이고 call의 리턴값이 output이 된다.

In [17]:
# 라이브러리 및 데이터 구성하기

import tensorflow as tf
from tensorflow import keras
import numpy as np

# 데이터 구성부분
mnist = keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train=x_train[...,np.newaxis]
x_test=x_test[...,np.newaxis]

print(len(x_train), len(x_test))

60000 10000


In [18]:
# Subclassing 구현

class CustomModel(keras.Model):
    def __init__(self):
        super().__init__()
        self.conv1 = keras.layers.Conv2D(32, 3, activation='relu')
        self.conv2 = keras.layers.Conv2D(64, 3, activation='relu')
        self.flatten = keras.layers.Flatten()
        self.fc1 = keras.layers.Dense(128, activation='relu')
        self.fc2 = keras.layers.Dense(10, activation='softmax')
    
    def call(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        
        return(x)

model = CustomModel()

In [19]:
# 모델 학습 설정

model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test, y_test, verbose=2)

Epoch 1/5
1875/1875 [==============================] - 7s 3ms/step - loss: 0.1040 - accuracy: 0.9679
Epoch 2/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0348 - accuracy: 0.9889
Epoch 3/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0184 - accuracy: 0.9945
Epoch 4/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0140 - accuracy: 0.9954
Epoch 5/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0095 - accuracy: 0.9969
313/313 - 1s - loss: 0.0365 - accuracy: 0.9903


[0.03650324419140816, 0.9902999997138977]

### Sequential API를 활용해서 CIFAR-100 문제 해결

In [20]:
# 라이브러리 및 데이터 구성

import tensorflow as tf
from tensorflow import keras

# 데이터 구성부분
cifar100 = keras.datasets.cifar100

(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
print(len(x_train), len(x_test))

169017344/169001437 [==============================] - 3s 0us/step
50000 10000


In [23]:
# Sequential Model 구성

model = keras.Sequential([
    keras.layers.Conv2D(16, 3, activation='relu'),
    keras.layers.MaxPool2D((2,2)),
    keras.layers.Conv2D(32, 3, activation='relu'),
    keras.layers.MaxPool2D((2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dense(100, activation='softmax')
])

In [24]:
# 모델 학습 설정

model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test, y_test, verbose=2)

Epoch 1/5
1563/1563 [==============================] - 5s 3ms/step - loss: 3.6482 - accuracy: 0.1536
Epoch 2/5
1563/1563 [==============================] - 5s 3ms/step - loss: 2.9615 - accuracy: 0.2754
Epoch 3/5
1563/1563 [==============================] - 5s 3ms/step - loss: 2.6756 - accuracy: 0.3279
Epoch 4/5
1563/1563 [==============================] - 5s 3ms/step - loss: 2.4715 - accuracy: 0.3705
Epoch 5/5
1563/1563 [==============================] - 5s 3ms/step - loss: 2.2991 - accuracy: 0.4068
313/313 - 1s - loss: 2.6613 - accuracy: 0.3465


[2.661252498626709, 0.3465000092983246]

### Functional API 활용

In [25]:
# 라이브러리 및 데이터 구성

import tensorflow as tf
from tensorflow import keras

# 데이터 구성부분
cifar100 = keras.datasets.cifar100

(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
print(len(x_train), len(x_test))

50000 10000


In [27]:
# Functional API 구성

inputs = keras.Input(shape=(32, 32, 3))

x = keras.layers.Conv2D(16, 3, activation='relu')(inputs)
x = keras.layers.MaxPool2D((2,2))(x)
x = keras.layers.Conv2D(32, 3, activation='relu')(x)
x = keras.layers.MaxPool2D((2,2))(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(256, activation='relu')(x)
predictions = keras.layers.Dense(100, activation='softmax')(x)

model = keras.Model(inputs=inputs, outputs=predictions)

# 모델 학습 설정

model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test, y_test, verbose=2)

Epoch 1/5
1563/1563 [==============================] - 5s 3ms/step - loss: 3.6221 - accuracy: 0.1546
Epoch 2/5
1563/1563 [==============================] - 5s 3ms/step - loss: 2.9468 - accuracy: 0.2761
Epoch 3/5
1563/1563 [==============================] - 5s 3ms/step - loss: 2.6223 - accuracy: 0.3417
Epoch 4/5
1563/1563 [==============================] - 5s 3ms/step - loss: 2.4018 - accuracy: 0.3870
Epoch 5/5
1563/1563 [==============================] - 5s 3ms/step - loss: 2.2332 - accuracy: 0.4231
313/313 - 1s - loss: 2.5831 - accuracy: 0.3583


[2.58314847946167, 0.35830000042915344]

### Subclassing 활용

In [2]:
# 라이브러리 및 데이터 구성

import tensorflow as tf
from tensorflow import keras

# 데이터 구성부분
cifar100 = keras.datasets.cifar100

(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
print(len(x_train), len(x_test))

50000 10000


In [3]:
# Subclassing 구현

class CustomModel(keras.Model):
    def __init__(self):
        super().__init__()
        self.conv1 = keras.layers.Conv2D(16, 3, activation='relu')
        self.max1 = keras.layers.MaxPool2D((2,2))
        self.conv2 = keras.layers.Conv2D(32, 3, activation='relu')
        self.max2 = keras.layers.MaxPool2D((2,2))
        self.flatten = keras.layers.Flatten()
        self.fc1 = keras.layers.Dense(256, activation='relu')
        self.fc2 = keras.layers.Dense(100, activation='softmax')
    
    def call(self, x):
        x = self.conv1(x)
        x = self.max1(x)
        x = self.conv2(x)
        x = self.max2(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        
        return(x)

model = CustomModel()

# 모델 학습 설정

model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test, y_test, verbose=2)

Epoch 1/5
1563/1563 [==============================] - 30s 3ms/step - loss: 3.6243 - accuracy: 0.1532
Epoch 2/5
1563/1563 [==============================] - 5s 3ms/step - loss: 2.9196 - accuracy: 0.2807
Epoch 3/5
1563/1563 [==============================] - 5s 3ms/step - loss: 2.6252 - accuracy: 0.3418
Epoch 4/5
1563/1563 [==============================] - 5s 3ms/step - loss: 2.4259 - accuracy: 0.3829
Epoch 5/5
1563/1563 [==============================] - 5s 3ms/step - loss: 2.2572 - accuracy: 0.4178
313/313 - 1s - loss: 2.5945 - accuracy: 0.3560


[2.5945024490356445, 0.35600000619888306]